In [1]:
#pip install pdfplumber

In [3]:
import glob #getting file list
import os #working with filepaths
import pdfplumber # reading pdf
import shutil #for copying
import re #for searchin inside strings

#Build filelist of all PDF Files in the Path
filelist=glob.glob('*.pdf')

#Iterate through all PDF Files within filelist

for file_name in filelist:
    month = ''
    year = ''
    #print(file_name)  #Debugging
    with pdfplumber.open(file_name) as pdf:
        first_page = pdf.pages[0]
        first_page_text = first_page.extract_text()
        #print(first_page.extract_text() #Debugging
    
    #Specify different States for different PDF File Types and set them to False by Default    
    entgelt_state = False
    zusatzinfo_state = False
    lohnsteuer_state = False
    sozversicherung_state = False
    kk_anmeldung_state = False
    kk_abmeldung_state = False
    #Default Value for Dateityp
    Dateityp = 'DUMMY'
    
    #Check First State Entgeltdokument
    #Has Month and Year in it
    
    if entgelt_state == False:
        
        try:
            found = re.search('Entgeltabrechnung für (.+?)             ', first_page_text).group(1)
            month, year = found.rsplit(" ", 1)
            entgelt_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
    
    #Check Second State Zusatzinformation für Entgelt
    #Has Month and Year in it
    
    if zusatzinfo_state == False and entgelt_state == False:
        try:
            found = re.search('Zusatzinformationen zur Entgeltabr für (.+?)  ', first_page_text).group(1)
            month, year = found.rsplit(" ", 1)
            zusatzinfo_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
    #Check Third State Lohnsteuerbescheid
    #Has only Year in it
    
    if zusatzinfo_state == False and entgelt_state == False and lohnsteuer_state == False:
        try:
            year = re.search('Ausdruck der elektronischen Lohnsteuerbescheinigung für (.+?)\n', first_page_text).group(1)
            #print('Found year is: ',year)
            lohnsteuer_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
            
    if zusatzinfo_state == False and entgelt_state == False and lohnsteuer_state == False and sozversicherung_state == False:
        try:
            soz_range = re.search('Jahresmeldung vom (.+?)\n', first_page_text).group(1)
            start_soz, end_soz = soz_range.rsplit(" bis ", 1)
            year = start_soz.rsplit(".", 1)[1]
            start_soz = start_soz.replace(".", "")
            end_soz = end_soz.replace(".", "")
            #print('Found year is: ',soz_range)
            #print('Start is: ',start_soz, ' End is: ', end_soz)
            sozversicherung_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
    
    if zusatzinfo_state == False and entgelt_state == False and lohnsteuer_state == False and sozversicherung_state == False and kk_anmeldung_state == False:
        try:
            kk_wechsel_date = re.search('Anmeldung \(Krankenkassenwechsel\) zum (.+?)\n', first_page_text).group(1)
            #start_soz, end_soz = soz_range.rsplit(" bis ", 1)
            year = kk_wechsel_date.rsplit(".", 1)[1]
            kk_wechsel_date = kk_wechsel_date.replace(".", "")
            #print('Found year is: ',year)
            #print('Start is: ' , kk_wechsel_date)
            kk_anmeldung_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
    if zusatzinfo_state == False and entgelt_state == False and lohnsteuer_state == False and sozversicherung_state == False and kk_anmeldung_state == False and kk_abmeldung_state == False:
        try:
            kk_wechsel_date = re.search('Abmeldung \(Krankenkassenwechsel\) vom (.+?)\n', first_page_text).group(1)
            start_abmeldung, end_abmeldung = kk_wechsel_date.rsplit(" bis ", 1)
            year = kk_wechsel_date.rsplit(".", 1)[1]
            start_abmeldung = start_abmeldung.replace(".", "")
            end_abmeldung = end_abmeldung.replace(".", "")
            #print('start_abmeldung year is: ',start_abmeldung)
            #print('end_abmeldung is: ' , end_abmeldung)
            kk_abmeldung_state = True
        except AttributeError:
            # AAA, ZZZ not found in the original string
            found = 'NOT FOUND' # apply your error handling
     
    #Debugging
    #print('entgelt_state= ', entgelt_state)
    #print('zusatzinfo_state= ', zusatzinfo_state)
    #print('lohnsteuer_state= ', lohnsteuer_state)
    #print('sozversicherung_state= ', sozversicherung_state)
    #print('kk_anmeldung_state= ', kk_anmeldung_state)
    #print('kk_abmeldung_state= ', kk_abmeldung_state)
    
    
    #print(found) #Debugging
    #print('Found month: ',month) #Debugging
    #print('Found year: ',year) #Debugging
    
    #Statemachine for Specifying the Month String to Month ID
    if month=='Januar':
        month_id='01'
    elif month=='Februar':
        month_id='02'
    elif month=='März':
        month_id='03'
    elif month=='April':
        month_id='04'
    elif month=='Mai':
        month_id='05'
    elif month=='Juni':
        month_id='06'
    elif month=='Juli':
        month_id='07'
    elif month=='August':
        month_id='08'
    elif month=='September':
        month_id='09'
    elif month=='Oktober':
        month_id='10'
    elif month=='November':
        month_id='11'
    elif month=='Dezember':
        month_id='12'

   

    # Check whether the specified path exists or not
    isExist = os.path.exists(year)
    #print(isExist)
    
    if year != '':
        if not isExist:
            # Create a new directory because it does not exist 
            os.makedirs(year)
            #print("The new directory is created!")
            
    #Statemachine for putting File destination and new Name together 
    if entgelt_state == True:
        Dateityp = 'Entgeltabrechnung_'
        dst=year+'/'+ Dateityp +year+'_'+month_id+'.pdf'
        shutil.copy(file_name,dst)
    elif zusatzinfo_state == True:
        Dateityp = 'Zusatzinformationen_'
        dst=year+'/'+ Dateityp +year+'_'+month_id+'.pdf'
        shutil.copy(file_name,dst)
    elif lohnsteuer_state == True:
        Dateityp = 'Elektronische_Lohnsteuerbescheinigung_'
        dst=year+'/'+ Dateityp +year+'.pdf'
        shutil.copy(file_name,dst)
    elif sozversicherung_state == True:
        Dateityp = 'Meldebescheinigung_Sozialversicherung_'
        dst=year+'/'+ Dateityp + start_soz + '_' + end_soz + '.pdf'
        shutil.copy(file_name,dst)
    elif kk_anmeldung_state == True:
        Dateityp = 'Meldebescheinigung_Sozialversicherung_'
        dst=year+'/'+ Dateityp + kk_wechsel_date + '.pdf'
        shutil.copy(file_name,dst)
    elif kk_abmeldung_state == True:
        Dateityp = 'Meldebescheinigung_Sozialversicherung_'
        dst=year+'/'+ Dateityp + start_abmeldung + '_' + end_abmeldung + '.pdf'
        shutil.copy(file_name,dst)   
    
    #print('________________________________________________________________________________________')
        
    

 



